In [ ]:
import pandas as pd
import os

# Función para cargar datos desde un archivo CSV
def cargar_datos_desde_csv(archivo_csv):
    """
    Carga los datos desde un archivo CSV.

    Args:
        archivo_csv (str): Ruta al archivo CSV.

    Returns:
        DataFrame: Datos cargados desde el archivo CSV.
    """
    data = pd.read_csv(archivo_csv)
    return data

# Función para asignar un perfil a partir de respuestas
def asignar_perfil(respuestas, perfiles, puntos_perfil):
    """
    Asigna un perfil de usuario a partir de las respuestas.

    Args:
        respuestas (list): Lista de respuestas del usuario.
        perfiles (list): Lista de perfiles disponibles.
        puntos_perfil (dict): Diccionario de puntos por perfil.

    Returns:
        str: Perfil asignado.
    """
    puntuaciones = {perfil: sum(a * b for a, b in zip(respuestas, puntos_perfil[perfil])) for perfil in perfiles}
    perfil_asignado = max(puntuaciones, key=puntuaciones.get)
    return perfil_asignado

# Función para asignar perfiles a todos los archivos CSV en el directorio actual
def asignar_perfiles_a_todos_csv(preguntas, perfiles, puntos_perfil):
    """
    Asigna perfiles de usuario a todos los archivos CSV en el directorio actual.

    Args:
        preguntas (list): Lista de nombres de preguntas.
        perfiles (list): Lista de perfiles disponibles.
        puntos_perfil (dict): Diccionario de puntos por perfil.

    Returns:
        None
    """
    archivos_csv = [archivo for archivo in os.listdir() if archivo.endswith('.csv')]
    
    for archivo_csv in archivos_csv:
        data = cargar_datos_desde_csv(archivo_csv)

        data['Perfil'] = data[preguntas].apply(lambda row: asignar_perfil(row, perfiles, puntos_perfil), axis=1)

        data.to_csv(archivo_csv, index=False)

# Ejemplo de uso:
preguntas = ["Pregunta1", "Pregunta2", "Pregunta3", "Pregunta4", "Pregunta5", "Pregunta6", "Pregunta7", "Pregunta8", "Pregunta9", "Pregunta10"]
perfiles = ["Turista de Playa", "Turista de Ciudades", "Aventurero", "Cultura y Arte", "Gastronómico"]
puntos_perfil = {
    "Turista de Playa": [5, 1, 2, 1, 1, 4, 1, 3, 2, 2],
    "Turista de Ciudades": [1, 5, 1, 3, 2, 2, 2, 3, 3, 3],
    "Aventurero": [2, 2, 5, 1, 1, 3, 2, 5, 1, 2],
    "Cultura y Arte": [2, 2, 1, 5, 1, 1, 5, 3, 2, 2],
    "Gastronómico": [1, 2, 2, 2, 5, 1, 2, 2, 1, 1]
}

asignar_perfiles_a_todos_csv(preguntas, perfiles, puntos_perfil)


In [ ]:
from flask import Flask, request, jsonify
import os
import pandas as pd

app = Flask(__name)

# Función para cargar datos desde un archivo CSV
def cargar_datos_desde_csv(archivo_csv):
    data = pd.read_csv(archivo_csv)
    return data

# Función para asignar un perfil a partir de respuestas
def asignar_perfil(respuestas, perfiles, puntos_perfil):
    puntuaciones = {perfil: sum(a * b for a, b in zip(respuestas, puntos_perfil[perfil])) for perfil in perfiles}
    perfil_asignado = max(puntuaciones, key=puntuaciones.get)
    return perfil_asignado

@app.route('/cargar_csv', methods=['POST'])
def cargar_csv():
    if 'file' not in request.files:
        return jsonify({'error': 'No se proporcionó un archivo CSV'})

    archivo = request.files['file']

    if archivo.filename == '':
        return jsonify({'error': 'Nombre de archivo no válido'})

    if archivo:
        archivo.save(archivo.filename)
        return jsonify({'message': 'Archivo CSV cargado exitosamente'})

@app.route('/asignar_perfiles', methods=['POST'])
def asignar_perfiles():
    archivos_csv = [archivo for archivo in os.listdir() if archivo.endswith('.csv')]
    if not archivos_csv:
        return jsonify({'error': 'No se encontraron archivos CSV en el directorio'})

    preguntas = request.json.get('preguntas')
    perfiles = request.json.get('perfiles')
    puntos_perfil = request.json.get('puntos_perfil')

    for archivo_csv in archivos_csv:
        data = cargar_datos_desde_csv(archivo_csv)

        data['Perfil'] = data[preguntas].apply(lambda row: asignar_perfil(row, perfiles, puntos_perfil), axis=1)

        data.to_csv(archivo_csv, index=False)

    return jsonify({'message': 'Perfiles asignados exitosamente'})

@app.route('/listar_csv', methods=['GET'])
def listar_csv():
    archivos_csv = [archivo for archivo in os.listdir() if archivo.endswith('.csv')]
    return jsonify({'archivos_csv': archivos_csv})

if __name__ == '__main__':
    app.run(debug=True)
